In [75]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice 
from azureml.core.model import Model
from azureml.core import Workspace, Experiment


# ws = Workspace.get(name="capstone")
ws = Workspace.from_config()
# exp = Experiment(workspace=ws, name="udacity-project-hyperparam-bayes")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [76]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

# env = Environment.from_conda_specification(name = 'project1', file_path = '.azureml/project1.yml')
# env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)
env = Environment.get(workspace=ws, name="AzureML-AutoML")
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True)

In [77]:
model = Model(ws, name='automl')
service = Model.deploy(workspace=ws,
                        name = 'automl-auth',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=deployment_config
)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://d95e931a-f942-4a3c-b8ca-649d7e5607ed.westeurope.azurecontainer.io/score


## Get testset

In [28]:
import pandas as pd
from sklearn.impute import SimpleImputer
train_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/housing_after_preprocessing.csv"
train_x = pd.read_csv(train_url)
train_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
train_x = train_x.drop(columns=['y'])

class DFSimpleImputer(SimpleImputer):
    # just like SimpleImputer, but retuns a df
    # this approach creates problems with the add_indicator=True, since more columns are returned
    # so don't set add_indicator to True
    def transform(self, X,y=None):
        return pd.DataFrame(super().transform(X),columns=X.columns) 
    def __repr__(self):
        return f'SimpleImputer'
imp = DFSimpleImputer(strategy='most_frequent')
imp = imp.fit(train_x)

In [29]:
test_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"
test_x = pd.read_csv(test_url)
test_x.rename(columns={'Unnamed: 0':'Column1'}, inplace=True)
test_x = imp.transform(test_x)
test_x = test_x.astype(train_x.dtypes.to_dict())

In [30]:
def submit(model, filename='submission.csv'):
    pred = model.predict(test_x)
    out = pd.DataFrame()
    out['Id']= list(range(1461, 2920))
    out['SalePrice'] = np.exp(pred)
    out.to_csv('./submissions/'+ filename, index=False)


In [31]:
from azureml.core import Workspace, Experiment, Model
import joblib
model = Model(workspace=ws, name='automl')
model.download(target_dir='models/automl', exist_ok = True)


'models\\automl\\outputs'

In [32]:
m = joblib.load('models/automl/outputs/model.pkl')

In [42]:
data = json.dumps(test_x.to_json())
data = json.loads(data)
data = pd.read_json(data)
res=m.predict(data)

In [80]:
service.scoring_uri

'http://d95e931a-f942-4a3c-b8ca-649d7e5607ed.westeurope.azurecontainer.io/score'

In [81]:
import pandas as pd
import json
import requests 
key = 'HZEYt6ai2g8DjRjBe1cliQuVtPt6EFAO'

headers = {'Content-Type':'application/json'}
headers['Authorization'] = f'Bearer {key}'
data = json.dumps(test_x.to_json())
# data = json.loads(data)
response = requests.post(service.scoring_uri, data=data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())


401
0:00:00.053420


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [79]:
input_data = json.dumps(test_x.to_json())
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'


# Make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
print(resp.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [66]:
min(resp.json()['predictions'])

10.753366289948861

In [ ]:

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://b3b6303a-12d0-4ee1-b703-11604fab8086.westeurope.azurecontainer.io/score'

# If the service is authenticated, set the key or token
key = 'HZEYt6ai2g8DjRjBe1cliQuVtPt6EFAO'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 87,
            "campaign": 2,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "tue",
            "default": "no",
            "duration": 971,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 10099.1,
            "pdays": 999,
            "poutcome": "success",
            "previous": 1
          },
          {
            "age": 87,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 471,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())